In [1]:
import pandas as pd
import numpy as np
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import StratifiedKFold
from osgeo import gdal

In [2]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import StratifiedKFold

def prepare_data(X_res, y_res):
    """
    Scales the input data X_res and returns it along with y_res.
    
    Parameters:
        X_res : DataFrame
            Feature data.
        y_res : DataFrame
            Target data.
    """
    y = y_res['class']
    X_scaled = StandardScaler().fit_transform(X_res)
    return X_scaled, y

def prepare_data_f(file_name):
    """
    Reads a CSV file, extracts features and target variable, scales the feature data and returns the scaled data and target.
    
    Parameters:
        file_name : str
            Path to the input CSV file.
    """
    df = pd.read_csv(file_name)
    y = df['class']
    X = df.drop(['E', 'N', 'class'], axis=1)
    X_scaled = StandardScaler().fit_transform(X)
    return X_scaled, y

def train_model(X, y):
    """
    Trains a MLP classifier using StratifiedKFold for cross-validation.
    
    Parameters:
        X : array-like of shape (n_samples, n_features)
            Sample data.
        y : array-like of shape (n_samples,)
            Target values.
    
    Returns:
        clf : MLPClassifier
            Trained model.
    """
    skf = StratifiedKFold(n_splits=5)
    clf = MLPClassifier(solver='adam', alpha=.001, hidden_layer_sizes=(16, 2), random_state=1)

    for train_index, test_index in skf.split(X, y):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        clf.fit(X_train, y_train)
        print(f"Accuracy: {clf.score(X_test, y_test)}")
    
    return clf

def process_data(file_name, clf, threshold):
    """
    Function to process data, generate probabilities, and save to CSV.

    :param file_name: string, name of the input CSV file.
    :param clf: classifier object, to predict probabilities.
    :param threshold: float, threshold value for class assignment.
    """
    
    # Load data
    df0 = pd.read_csv(file_name)
    y0 = df0['class']
    df0 = df0.drop(['E', 'N', 'class'], axis=1)
    
    # Standardize data
    df0 = StandardScaler().fit_transform(df0)

    df0_numpy = df0.copy()

    # Ensure data is in NumPy array format
    if not isinstance(df0, np.ndarray):
        df0_numpy = df0.to_numpy()

    df0 = df0_numpy.copy()
    
    # Calculate the probability of each point
    probs1 = clf.predict_proba(df0)[:, 0]
    probs2 = clf.predict_proba(df0)[:, 1]

    print(len(probs2))
    count = np.count_nonzero(probs2>threshold)
    print(count)


    for i in range(len(probs2)):
      if probs2[i] > threshold:
        probs2[i] = 1
      else:
        probs2[i] = 0

    # Apply threshold and convert probabilities to class labels
    # probs2 = np.where(probs2 > threshold, 1, 0).astype(int)
    
    # Reload original data
    df0_original = pd.read_csv(file_name)
    
    # Create new dataframe with original E, N columns and new probability columns
    new_df = df0_original.loc[:, ['E', 'N']].assign(Probability1=probs2, Probability2=probs1)
    
    # Save new data to CSV
    new_df.to_csv("probability1.csv")

    # Output for user
    print(f"Processed data saved to 'probability.csv'")
    print(f"Data shape: {new_df.shape}")
    print(new_df.head())
    return new_df

In [3]:
# Example usage:

# Load the CSV file

# Load the independent variables (features) from a CSV file into a DataFrame.
# Assuming 'X_res.csv' contains the feature variables used for training the model.
X_train = pd.read_csv('/home/dipak/Desktop/codes/Probability MLP/X_res.csv')

# Load the dependent variable (target) from a CSV file into a DataFrame.
# Assuming 'y_res.csv' contains the target variable corresponding to X_train.
y_train = pd.read_csv('/home/dipak/Desktop/codes/Probability MLP/y_res.csv')


# Define the path to the input data file.
# Assuming '2M_raster_points.csv' contains columns 'E', 'N', and some class label.
filename_input_csv = '/home/dipak/Desktop/codes/Probability MLP/2M_raster_points.csv'


# Prepare data and train model
X_train, y_train = prepare_data(X_train,y_train)

# Train the model using the prepared data.
clf = train_model(X_train, y_train)

Accuracy: 0.9999864393540793
Accuracy: 0.9999769469019348
Accuracy: 0.9998603251576088
Accuracy: 0.9992419588650809
Accuracy: 0.9999742347378113


In [4]:
# Define a threshold for class probability. 
# Any predicted probability above this value might be classified as class 1, otherwise class 0.
# It's commonly used to determine the decision boundary in probabilistic classifiers.
threshold = 0.9

# Prepare Data for Probability Calculation
X_prob, y_prob = prepare_data_f(filename_input_csv)

# Process Data and Save Probabilities
new_df = process_data(filename_input_csv, clf, threshold)

1843564
27449
Processed data saved to 'probability.csv'
Data shape: (1843564, 4)
          E          N  Probability1  Probability2
0  360300.0  7540050.0           0.0           1.0
1  360350.0  7540050.0           0.0           1.0
2  360400.0  7540050.0           0.0           1.0
3  360450.0  7540050.0           0.0           1.0
4  360500.0  7540050.0           0.0           1.0


,E,N,Probability1,Probability2
0,360300.0,7540050.0,0.0,1.0
1,360350.0,7540050.0,0.0,1.0
2,360400.0,7540050.0,0.0,1.0
3,360450.0,7540050.0,0.0,1.0
4,360500.0,7540050.0,0.0,1.0


In [5]:
def create_target_raster_with_csv_file(file_path, pixelwise_classification_options):
  
    """
    This function generates a dictionary containing information about the spatial extent
    and resolution of a target raster, based on point data extracted from a CSV file.
    
    Parameters:
    - file_path (str): Path to the CSV file containing point data with 'N' and 'E' columns, representing
                       spatial coordinates.
    - pixelwise_classification_options (dict): A dictionary containing options for raster generation,
                                               including "OVERRIDE_RASTER_SIZE" which represents the
                                               desired spatial resolution of the raster.
    
    Returns:
    - main_target (dict): A dictionary containing information about the target raster, including
                          its spatial extent, resolution, and dimension in pixels.
    """


    # Using pandas to load data from the CSV file at the given file_path. 
    # Assumes that the file contains columns 'N' and 'E' to represent spatial coordinates.
    ds = pd.read_csv(f'{file_path}')

    # Extract Spatial Coordinates from the CSV file
    # Extracting 'N' and 'E' coordinates as NumPy arrays from the DataFrame.
    list_of_all_N = ds['N'].values
    list_of_all_E = ds['E'].values

    # Compute Target Raster Attributes
    # --------------------------------
    # Calculating attributes of the target raster based on the spatial extent of the input points and the desired raster resolution defined in 
    # pixelwise_classification_options.
    # The 'main_target' dictionary will contain several attributes related to the spatial and 
    # pixel dimensions of the resultant raster, like min/max coordinates, raster width/height,
    # number of pixels in both dimensions, and spatial resolution in meters.

    main_target = {
                    'target_raster_max_y': np.max(list_of_all_N),
                    'target_raster_min_y': np.min(list_of_all_N),
                    'target_raster_min_x': np.min(list_of_all_E),
                    'target_raster_max_x': np.max(list_of_all_E),
                    'number_of_x_pixels': int(round(np.max(list_of_all_E) - np.min(list_of_all_E)) / float(pixelwise_classification_options["OVERRIDE_RASTER_SIZE"])),
                    'number_of_y_pixels': int(round(np.max(list_of_all_N) - np.min(list_of_all_N)) / float(pixelwise_classification_options["OVERRIDE_RASTER_SIZE"])),
                    'raster_height': np.max(list_of_all_N) - np.min(list_of_all_N),
                    'raster_width': np.max(list_of_all_E) - np.min(list_of_all_E),
                    'target_raster_spatial_x_pixel_size_meters': pixelwise_classification_options["OVERRIDE_RASTER_SIZE"],
                    'target_raster_spatial_y_pixel_size_meters': pixelwise_classification_options["OVERRIDE_RASTER_SIZE"],
                    }

    return main_target


In [6]:
# This file is assumed to contain the calculated probability data from the previous step for subsequent spatial processing.
file_path = '/home/dipak/Desktop/Final_File/MLP_Probability/probability1.csv'
new_df = pd.read_csv(file_path)

# Here, "OVERRIDE_RASTER_SIZE" is set to 50, which will be used to determine the spatial resolution of the generated raster.
pixelwise_classification_options = {"OVERRIDE_RASTER_SIZE": 50}

# Generating Target Raster Attributes
# -----------------------------------
# Calling the previously defined function to calculate the attributes of the target raster based on point data from 'probability1.csv'.
# The returned 'mine_target' dictionary contains detailed information about the spatial extent and resolution of the desired raster.
mine_target = create_target_raster_with_csv_file('probability1.csv', pixelwise_classification_options)
print(mine_target)

# Raster Dimensions and Initialization
# -----------------------------------
# Extracting the desired number of pixels in each dimension (height and width) from the 'mine_target' dictionary, and defining a nodata_value which will 
# be used to initialize the raster array.
height_size_in_pixel = mine_target['number_of_y_pixels']
width_size_in_pixel = mine_target['number_of_x_pixels']
nodata_value = 255

# Creating an empty raster array, initializing all pixels to the nodata_value.
# The dtype 'uint8' signifies that the array will store unsigned 8-bit integer values.
array = np.full((height_size_in_pixel, width_size_in_pixel), nodata_value, dtype='uint8')

# Extracting Raster Origin and Resolution
# ---------------------------------------
# Extracting and printing the coordinates of the top-left corner (origin) of the raster ('current_east' and 'current_north'), and the spatial resolution 
# ('spatial_pixel_resolution') from the 'mine_target' dictionary.
# These values will be crucial for subsequent geospatial processing and analysis.
current_east = mine_target['target_raster_min_x']
current_north = mine_target['target_raster_max_y']
spatial_pixel_resolution = mine_target['target_raster_spatial_x_pixel_size_meters']
print(current_east, current_north)

{'target_raster_max_y': 7540050.0, 'target_raster_min_y': 7458100.0, 'target_raster_min_x': 346050.0, 'target_raster_max_x': 414000.0, 'number_of_x_pixels': 1359, 'number_of_y_pixels': 1639, 'raster_height': 81950.0, 'raster_width': 67950.0, 'target_raster_spatial_x_pixel_size_meters': 50, 'target_raster_spatial_y_pixel_size_meters': 50}
346050.0 7540050.0


In [7]:
import numpy as np
import pandas as pd

def analyze_and_assign_values(height_size_in_pixel, width_size_in_pixel, current_north, current_east, spatial_pixel_resolution, threshold, df, array):
    """
    Analyze given DataFrame to find specific values and assign updates to the array based on the conditions met.

    Parameters:
    - height_size_in_pixel (int): Total number of pixels in height to be analyzed.
    - width_size_in_pixel (int): Total number of pixels in width to be analyzed.
    - current_north (float): Current northern value to begin analysis.
    - current_east (float): Current eastern value to begin analysis.
    - spatial_pixel_resolution (float): The spatial resolution of a pixel.
    - threshold (float): Threshold value to decide whether to update the array or not.
    - df (DataFrame): A DataFrame containing data to be analyzed. Expecting columns ['E', 'N'].
    - array (ndarray): A NumPy array to be updated based on the analysis.

    Returns:
    - ndarray: Updated array based on the analysis.
    """

    # Loop through each pixel in height and width.
    for R in range(height_size_in_pixel):
        # Calculate the point in the North to be analyzed by decrementing the spatial resolution for each step.
        point_N_to_analyse = current_north - (R * spatial_pixel_resolution)
        for C in range(width_size_in_pixel):
            # Calculate the point in the East to be analyzed by incrementing the spatial resolution for each step.
            point_E_to_analyse = current_east + (C * spatial_pixel_resolution)
            
            # Fetch the needed values from the DataFrame where the East and North match the points being analyzed.
            value_I_need = df.loc[(df['E'] == point_E_to_analyse) & (df['N'] == point_N_to_analyse)].values
            
            # If the fetched values are non-empty and meet the threshold, update the array and print the value.
            if len(value_I_need) != 0 and value_I_need[0][3] * 100 >= threshold * 100:
                print(value_I_need[0][3] * 100)
                array[R, C] = 1

    return array


In [8]:
# Value Analysis and Array Update
# Calling the 'analyze_and_assign_values' function to analyze spatial data and assign appropriate values to the raster array based on certain conditions.
updated_array = analyze_and_assign_values(height_size_in_pixel, width_size_in_pixel, current_north, current_east, spatial_pixel_resolution, threshold, new_df, array)

100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.

In [11]:
# save the Updated array csv
np.savetxt("updated_array", updated_array, delimiter=",")


In [9]:
def openTiffFileAsRaster(filename):
    """
    Open a TIFF file and return its data and dataset object.
    
    Parameters:
        filename (str): Path to the TIFF file.
        
    Returns:
        arr (ndarray): Array containing raster data.
        ds (Dataset): GDAL dataset object.
    """
    ds = gdal.Open(filename)
    band = ds.GetRasterBand(1)
    arr = band.ReadAsArray()
    return arr, ds

def saveRasterToTiffFile(arr, ds, filename, geotrans, raster_output_type, nodata_value):
    """
    Save raster data to a TIFF file with specified properties.
    
    Parameters:
        arr (ndarray): Raster data to save.
        ds (Dataset): Reference GDAL dataset object.
        filename (str): Path to save the output TIFF file.
        geotrans (tuple): GeoTransform parameters.
        raster_output_type (str): Type of raster output ("classification" or other).
        nodata_value (int/float): No-data value for the output raster.
    """
    [cols, rows] = arr.shape
    driver = gdal.GetDriverByName("GTiff")
    
    outdata = None
    if raster_output_type == "classification":
        outdata = driver.Create(filename, rows, cols, 1, gdal.GDT_Byte)
    else: # Regression on default.
        outdata = driver.Create(filename, rows, cols, 1, gdal.GDT_Float32)
    # outdata.SetGeoTransform(ds.GetGeoTransform())##sets same geotransform as input
    outdata.SetGeoTransform(geotrans)
    # sets same geotransform as input
    outdata.SetProjection(ds.GetProjection())
    # sets same projection as input
    outdata.GetRasterBand(1).WriteArray(arr)
    outdata.GetRasterBand(1).SetNoDataValue(nodata_value)
    # if you want these values transparent
    outdata.FlushCache()
    # saves to disk!!
    outdata = None
    band=None
    ds=None


In [10]:
from osgeo import gdal

# Extracting and assigning specific properties from the 'mine_target' dictionary, which contains properties related to a target raster, such as 
# its extents and spatial resolution.
target_raster_min_E = mine_target['target_raster_min_x']
target_raster_max_N = mine_target['target_raster_max_y']
target_raster_spatial_x_pixel_size_meters = mine_target['target_raster_spatial_x_pixel_size_meters']

# Assigning the spatial resolution of a pixel to a more concise variable for use in further operations.
# The spatial resolution denotes the size of a pixel in real-world units (here, meters).
pixel_spatial_resolution = target_raster_spatial_x_pixel_size_meters


# Defining the path to a reference data file and specifying the type of raster output desired.
# reference_data_path: Path to a TIFF file that presumably contains reference raster data.
# raster_output_type: Desired output type for a raster file, here set to "classification".
reference_data_path = "/home/dipak/Desktop/codes/Probability MLP/AEM/IOCG_AEM_Inph_.tif"
raster_output_type = "classification"

# Geo-Transform Information
# Defining geotransform information for positioning the raster in the coordinate space.
# Geotransform contains:
# [0] top-left x-coordinate (min E)
# [1] width of a pixel (spatial resolution in the x-direction, in meters)
# [2] rotation term (0 if image is oriented north up)
# [3] top-left y-coordinate (max N)
# [4] rotation term (0 if image is oriented north up)
# [5] height of a pixel (negative spatial resolution in the y-direction, in meters)
geotrans_info = (target_raster_min_E, pixel_spatial_resolution, 0.0, target_raster_max_N, 0.0, -pixel_spatial_resolution)
projection_arr, projection_reference_info_ds = openTiffFileAsRaster(reference_data_path)

# Saving the Updated Raster to a TIFF File
saveRasterToTiffFile(arr=updated_array.astype('uint8'), 
                        ds=projection_reference_info_ds,
                        filename="dipak_321.tif", 
                        geotrans=geotrans_info, 
                        raster_output_type="classification",
                        nodata_value=255
                        )
